На основе регламента контроля и взыскания дебиторской задолженности создайте нейро-ассистента для ответов на вопросы сотрудников по данной базе знаний. Документ с базой знаний можно найти по ссылке: https://docs.google.com/document/d/1UD9lSQy6PcCejUy3k5ARdUBNS-GEhXqOjnMLMZ6r488/edit?usp=sharing

Создайте нейро-ассистента на основе материала по теме. Напишите инструкцию в system. Проверьте работу нейро-ассистента на нескольких самостоятельно сформулированных вопросах.

In [ ]:
# установка библиотек

!pip install faiss-cpu==1.8.0 openai==1.55.3 tiktoken==0.7.0 langchain_openai==0.1.22 langchain_community==0.2.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully un

In [ ]:
# импорт библиотек

import os
import getpass
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [ ]:
# настройка ключа OPENAI_API

openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# Загружаем база знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1UD9lSQy6PcCejUy3k5ARdUBNS-GEhXqOjnMLMZ6r488/edit?usp=sharing')
data_from_url


'\ufeffРегламент контроля и взыскания дебиторской задолженности\r\nРегламент контроля дебиторской задолженности (далее ДЗ) и взыскания просроченной ДЗ (далее ПДЗ)\r\n1. Назначение Регламента.\r\nНастоящий Регламент предназначен для:\r\n* Системного предупреждения возникновения просроченной дебиторской задолженности (далее – ПДЗ), предотвращения возникновения невозвратной и хронической дебиторской задолженности.\r\n* Сокращения среднего времени просрочек и уменьшения среднего объема ПДЗ.\r\n* Удержания размеров ПДЗ в пределах планово-нормативных значений.\r\n2. Область применения настоящего Регламента.\r\n2.1 Настоящий Регламент применяется ко всем клиентам, работающим с отсрочкой платежей и ко всем случаям возникновения ДЗ и ПДЗ, независимо от категорий клиентов, кроме случаев по которым существует письменное распоряжение руководства о полном или частичном исключении их из области применения настоящего Регламента.\r\n2.2 Регламент вступает в силу с момента возникновения у клиента ДЗ пе

In [ ]:
# разбиваем текстовый файл на чанки сплиттером от langchain
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))

source_chunks

[Document(page_content='\ufeffРегламент контроля и взыскания дебиторской задолженности\r\nРегламент контроля дебиторской задолженности (далее ДЗ) и взыскания просроченной ДЗ (далее ПДЗ)\r\n1. Назначение Регламента.\r\nНастоящий Регламент предназначен для:\r\n* Системного предупреждения возникновения просроченной дебиторской задолженности (далее – ПДЗ), предотвращения возникновения невозвратной и хронической дебиторской задолженности.\r\n* Сокращения среднего времени просрочек и уменьшения среднего объема ПДЗ.\r\n* Удержания размеров ПДЗ в пределах планово-нормативных значений.\r\n2. Область применения настоящего Регламента.\r\n2.1 Настоящий Регламент применяется ко всем клиентам, работающим с отсрочкой платежей и ко всем случаям возникновения ДЗ и ПДЗ, независимо от категорий клиентов, кроме случаев по которым существует письменное распоряжение руководства о полном или частичном исключении их из области применения настоящего Регламента.\r\n2.2 Регламент вступает в силу с момента возник

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)                           # заполните поле необходимым значением

In [ ]:

# готовим функцию для поиска релевантных отрезков из базы знаний, обращение к OpenAI и возврат ответа.

def answer_index(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer


In [ ]:
# готовим самостоятельно промпт

system="Ты – консультант, ответь на вопрос клиента строго на основе предоставленной информации. Не добавляй ничего от себя и не раскрывай источник информации."

In [ ]:
# проверьте работу нейро-консультанта на нескольких самостоятельно-сформулированных вопросах

topic= "Кто такие неплательщики?"
ans=answer_index(system, topic, db)
ans

In [ ]:
topic= "Кто занимается присвоением категорий клиентам?"
ans=answer_index(system, topic, db)
ans

In [ ]:
topic= "Какие докуметы является основным рабочими документами по контролю динамики ДЗ/ПДЗ?"
ans=answer_index(system, topic, db)
ans